# Vamos a capturar el vídeo de nuestra webcam con OpenCV
https://medium.com/algoasylum/blink-detection-using-python-737a88893825

https://github.com/algoasylum/BlinkDetection

https://github.com/opencv/opencv

In [ ]:
#Importamos los paquetes necesarios (si no los tenemos instalados, debemos hacerlo previamente)
import cv2
import dlib
import math

In [ ]:
#Cálculo de la ratio de parpadeo

def midpoint(point1 ,point2):
    return (point1.x + point2.x)/2,(point1.y + point2.y)/2

In [ ]:
def euclidean_distance(point1 , point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [ ]:
def get_blink_ratio(eye_points, facial_landmarks):
    
    #Carga de los puntos
    corner_left  = (facial_landmarks.part(eye_points[0]).x, 
                    facial_landmarks.part(eye_points[0]).y)
    corner_right = (facial_landmarks.part(eye_points[3]).x, 
                    facial_landmarks.part(eye_points[3]).y)
    
    center_top    = midpoint(facial_landmarks.part(eye_points[1]), 
                             facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), 
                             facial_landmarks.part(eye_points[4]))

    #Cálculo de la distancia
    horizontal_length = euclidean_distance(corner_left,corner_right)
    vertical_length = euclidean_distance(center_top,center_bottom)
    ratio = horizontal_length / vertical_length
    return ratio

In [ ]:
BLINK_RATIO_THRESHOLD = 5.7

In [ ]:
#Para empezar a capturar la imagen de la webcam
cap = cv2.VideoCapture(0)

In [ ]:
#Nombramos la ventana que recogerá la imagen de la webcam una vez ejecutemos
cv2.namedWindow('BlinkDetector')

In [ ]:
#Nombramos el detector facil de dlib
detector = dlib.get_frontal_face_detector()

In [ ]:
#Detección de los ojos con las medidas del proyecto original
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
left_eye_landmarks  = [36, 37, 38, 39, 40, 41]
right_eye_landmarks = [42, 43, 44, 45, 46, 47]

In [ ]:
while True:
    #Si es correcto, capturamos
    retval, frame = cap.read()

    #Salida de la aplicación si no hay imagen 
    if not retval:
        print("Can't receive frame (stream end?). Exiting ...")
        break 

    #Conversión a escala de grises
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Detección facial
    faces,_,_ = detector.run(image = frame, upsample_num_times = 0, 
                       adjust_threshold = 0.0)

    #Detección de ojos
    for face in faces:
        
        landmarks = predictor(frame, face)

        #Cálculo de la ratio de parpadeo
        left_eye_ratio  = get_blink_ratio(left_eye_landmarks, landmarks)
        right_eye_ratio = get_blink_ratio(right_eye_landmarks, landmarks)
        blink_ratio     = (left_eye_ratio + right_eye_ratio) / 2
        if blink_ratio > BLINK_RATIO_THRESHOLD:
            #Cuando detecta un parpadeo, mostrará el texto en rojo de la siguiente línea
            cv2.putText(frame,"BLINK",(10,50), cv2.FONT_HERSHEY_SIMPLEX,
                        2,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('BlinkDetector', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

In [ ]:
#Para parar el proceso y cerrar la ventana
cap.release()
cv2.destroyAllWindows()